## Perceptual Loss

In [1]:
from losses.Losses import *
from PIL import Image
from torchvision import transforms

ModuleNotFoundError: No module named 'health_multimodal'

In [ ]:
#transform = transforms.ToTensor()
transform = transforms.Compose([
    transforms.Grayscale(num_output_channels=3),  # Convert to 3-channel grayscale
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [ ]:
test_image_path = "/home/max/Desktop/MLMI/data/mimic-cxr-jpg/files/p10/p10000032/s50414267/02aa804e-bde0afdd-112c0b34-7bc16630-4e384014.jpg"
gt_image_path = "/home/max/Desktop/MLMI/data/mimic-cxr-jpg/files/p10/p10000764/s57375967/096052b7-d256dc40-453a102b-fa7d01c6-1b22c6b4.jpg"
test_image = Image.open(test_image_path)
test_image = transform(test_image)
gt_image = Image.open(gt_image_path)
gt_image = transform(gt_image)
test_image = test_image.unsqueeze(0)  # Add a batch dimension
gt_image = gt_image.unsqueeze(0)      # Add a batch dimension
print(gt_image.shape)

In [ ]:
perceptual_loss = Loss('biovil_t')
# perceptual_loss = Loss('biovil')
# perceptual_loss = Loss('ark', reference_path=None)
# perceptual_loss = Loss('vgg')
# perceptual_loss = Loss('style_vgg')

In [ ]:
loss_0 = perceptual_loss(gt_image, gt_image)
loss_1 = perceptual_loss(gt_image, test_image)
loss_2 = perceptual_loss(test_image, test_image)

In [ ]:
print(f'gt_vs_gt: {loss_0}')
print(f'gt_vs_test: {loss_1}')
print(f'gt_vs_test: {loss_2}')

## Difussion Model

In [1]:
from models.DDPM import ContextUnet, DDPM
import numpy as np
import pytorch_lightning as pl
from torchvision.datasets import MNIST, ImageFolder
from torch.utils.data import DataLoader
from torchvision import models, transforms
from pytorch_lightning.loggers import TensorBoardLogger

/home/mamur/miniconda3/envs/adlm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
n_epoch = 100
n_T = 400 # 500
device = "cuda:0"
n_classes = 3
n_feat = 128 # 128 ok, 256 better (but slower)
image_size = 64
lrate = 1e-4
batch_size = 32
save_model = False

In [3]:
model = DDPM(nn_model=ContextUnet(in_channels=1, n_feat=n_feat, n_classes=n_classes), 
                          betas=(1e-4,0.01),
                          n_T=n_T, drop_prob=0.1)

In [4]:
model

DDPM(
  (nn_model): ContextUnet(
    (init_conv): ResidualConvBlock(
      (conv1): Sequential(
        (0): Conv2d(1, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): GELU(approximate='none')
      )
      (conv2): Sequential(
        (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): GELU(approximate='none')
      )
    )
    (down1): UnetDown(
      (model): Sequential(
        (0): ResidualConvBlock(
          (conv1): Sequential(
            (0): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
            (1): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
            (2): GELU(approximate='none')
          )
          (conv2): Sequential(
            (0): Conv2d(128, 128, kernel_size=(3, 